In [1]:
#This notebook is for a data pipeline to format our data for unet

#Data sent to data/ is from patients LUNG1-001 to LUNG1-040 and is just slices near the middle 50% of tumors
#Data sent to data2/ is from patients LUNG1-101 to LUNG1-140 and is slices from anywhere in the middle 3/4ths 
#  of the scan with a 1:1 ratio of with-vs-without a tumor.
#Data sent to data3/ is basically data1 with no mask, and zoom instead, and 3 layers instead of one saved for the images.
#Data sent to data4/ is set up the same way as data2, but with a 50% zoom in instead of a mask for areas ourside 
#  the lung.
#DataSet 5 is all the patients, no ROI mask, zoomed, 3 layer images, images taken from the middle 50% of tumors only.

In [1]:
#But first!  Import stuff.
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import time
import numpy as np
import matplotlib.pyplot as plt
#import torch
#import torch.nn as nn
#from kural_core.models import *
#from kural_core.data_processing import *

PATH = 'E:\\DicomData\\LUNG1\\NSCLC-Radiomics\\'

# Stuff for dicom data
import os
import pydicom
from pydicom.data import get_testdata_files
#import nrrd


from skimage import morphology
from skimage import measure
from skimage import io
from skimage.transform import resize
from sklearn.cluster import KMeans
from skimage.draw import polygon
from PIL import Image

# My stuff
import function as my

def norm_image(x):
    if x.min() == 0 and x.max() == 0:
        img = (x*255.9).astype(np.uint8)
    else:
        img = (((x-x.min())/(x.max()-x.min()))*255.9).astype(np.uint8)
    return img

def format_image(x):
    img = (x*255.9).astype(np.uint8)
    return img

# 50% zoom (512x512 -> 256x256)
def zoom_image(x):
    N=len(x)
    img = x[int(0.25*N):int(0.75*N),int(0.25*N):int(0.75*N)]
    return img

In [3]:
train_count = 0
test_count = 0
dataFlag = 4                     # 1 for data1, 2 for data2, 3 for data3, 4 for data4
display = False                  # also displays images with mask overlays for tumor slices. 
                                 # (warning: this creates so many images your kernal may crash)

patients = [p for p in os.listdir(PATH)]

#set up rand seed.  In loop each slice will have a chance to go to the test or training set (for now 75-25)
if dataFlag == 1:
    rand_seed = 42  #For data  patients 0:40
    startP = 0
    endP = 40
    OUT_PATH = 'E:\\DicomData\\unet\\data\\'
elif dataFlag == 2:
    rand_seed = 24   #For data2, patients 100:140
    startP = 100
    endP = 140
    OUT_PATH = 'E:\\DicomData\\unet\\data2\\'
elif dataFlag == 3:
    rand_seed = 12  #For data3, patients 0:40  (alt data1)
    startP = 0
    endP = 40
    OUT_PATH = 'E:\\DicomData\\unet\\data3\\'
elif dataFlag == 4:
    rand_seed = 6   #For data4, patients 100:140
    startP = 100
    endP = 140
    OUT_PATH = 'E:\\DicomData\\unet\\data4\\'
elif dataFlag == 5:
    rand_seed = 3   #For data4, patients 100:140
    startP = 0
    endP =  422
    OUT_PATH = 'E:\\DicomData\\unet\\data5\\'
percent_train = 0.75
np.random.seed(rand_seed)
start_time = time.time()
elapsed_time = time.time() - start_time



for patient in patients[startP:endP:1]:
    p_start_time = time.time()
    print("Patient: "+patient)
    print("  Loading Data...")
    #load patient data
    data = []
    #Todo: replace with my.loadPatient(patient)
    for s in os.listdir(PATH+patient+"\\"):
        dir2 = [d for d in os.listdir(PATH+patient+"\\"+s+"\\")]        
        #print("  " + s)
        folder = ""
        folderC = ""
        if len(dir2) > 1:
            #We need to check which folder has the slices vs the contours.  There is a bunch of slices, but 1 contour.
            if (len(os.listdir(PATH+patient+"\\"+s+"\\"+dir2[1]))) == 1:
                #print("    " + dir2[0])
                #print("    " + dir2[1])
                folder = PATH+patient+"\\"+s+"\\"+dir2[0]+"\\"
                folderC = PATH+patient+"\\"+s+"\\"+dir2[1]+"\\"
            else:
                #print("    " + dir2[1])
                #print("    " + dir2[0])
                folder = PATH+patient+"\\"+s+"\\"+dir2[1]+"\\"
                folderC = PATH+patient+"\\"+s+"\\"+dir2[0]+"\\"
            contour = my.loadContour(folderC)
        else: 
            #print("    " + dir2[0])
            folder = PATH+patient+"\\"+s+"\\"+dir2[0]+"\\"
            contour = []
        slices = my.loadScan(folder)
        data=[slices,contour]
    
    
    if dataFlag is 1 or dataFlag is 2:    
        print("  Masking around ROI...")
    mask_slices = []
    for s in data[0]:
        
        if dataFlag == 1 or dataFlag == 2:  
            mask_slices.append(my.maskLung(s.pixel_array))
        else:
            #Because we skip mask_slices, we do a bit of data processing here
            temp_arr = s.pixel_array
            temp_arr = temp_arr - np.min(temp_arr)
            temp_arr = temp_arr/4095
            mask_slices.append(temp_arr)
            
            
            
    print("  Creating Truth Masks...") 
    if dataFlag == 5:
        tumor,col = my.labelSimpleTumor(data)
    else:
        tumor,col = my.labelTumor(data)
    
    print("  Selecting Data...")
    t_slices = []                          # list of slices with tumor
    nt_slices = []                         # list of slices with no tumor
    all_slices = []                        # list of all slices
    for j in range(tumor.shape[-1]):
        if tumor[...,j].any() != 0:
            t_slices.append(j)
            
            if display:
                my.plotMask(data[0][j].pixel_array,mask_slices[j],tumor[...,j])   
        else:
            nt_slices.append(j)
        all_slices.append(j)
    
    # now lets pull out the middle 50% of each of these tumors, to avoid training on edge effects.
    # or something different for data2

    if dataFlag == 1 or dataFlag == 3:
        num = len(t_slices)
        data_we_really_care_about = t_slices[int(num/4):int(3*num/4):1]
    elif dataFlag == 5:
        num = len(t_slices)
        data_we_really_care_about = t_slices[int(num/2):int(num/2)+1:1]
    else:
        num = len(all_slices)
        data_we_care_about = all_slices[int(num/8):int(7*num/8):1]
        #modify t_slices and nt_slices to remove data cut by data_we_care_about
        t_slices = list(set(t_slices).intersection(data_we_care_about))
        nt_slices = list(set(nt_slices).intersection(data_we_care_about))
        
        # now from this we pick a subset of individual random tumor and no tumor slices
        # the subset will be a random number between 0 and len(t_slices)
        if len(t_slices) != 0:
            if len(t_slices) < len(nt_slices):
                n_pairs = np.random.randint(int(np.floor(len(t_slices)/2.)),len(t_slices))
            else:
                n_pairs = np.random.randint(int(np.floor(len(nt_slices)/2.)),len(nt_slices))
            t_temp = np.random.choice(t_slices, n_pairs, replace = False)
            nt_temp = np.random.choice(nt_slices, n_pairs, replace = False)
            data_we_really_care_about = np.concatenate((t_temp,nt_temp),axis=None)
        else: 
            data_we_really_care_about = []
        
        
    
    #format data for unet and output
    print("  Outputing data...")
    for j,slices in enumerate(mask_slices):
        if j in data_we_really_care_about:
            r = np.random.random()
            if r < percent_train:
                
                I8_im = np.stack([format_image(zoom_image(x)) for x in mask_slices[j-1:j+2]], axis = -1)
                im = Image.fromarray(I8_im)
                im.save(OUT_PATH+'train//image//'+str(train_count)+'.png')
                
                x = tumor[...,j]
                x = zoom_image(x)
                I8_im = norm_image(x)
                im = Image.fromarray(I8_im)
                im.save(OUT_PATH+'train//label//'+str(train_count)+'.png')
                
                train_count = train_count+1
            else:
                
                I8_im = np.stack([format_image(zoom_image(x)) for x in mask_slices[j-1:j+2]], axis = -1)
                im = Image.fromarray(I8_im)
                im.save(OUT_PATH+'test//image//'+str(test_count)+'.png')
                
                x = tumor[...,j]
                x = zoom_image(x)
                I8_im = norm_image(x)
                im = Image.fromarray(I8_im)
                im.save(OUT_PATH+'test//label//'+str(test_count)+'.png')
                
                test_count = test_count+1
    
    elapsed_time = time.time() - p_start_time
    print("  Patient "+patient+" completed in "+str(int(elapsed_time))+" seconds")
    print("  Moving on.")
    
print("")
print("")
print("Finished")
print("  Output "+str(train_count)+" training images and "+str(test_count)+" test images")
elapsed_time = time.time() - start_time
print("  Total time elapsed was "+str(int(elapsed_time))+" seconds")

Patient: LUNG1-101
  Loading Data...
  Creating Truth Masks...
  Selecting Data...
  Outputing data...
  Patient LUNG1-101 completed in 9 seconds
  Moving on.
Patient: LUNG1-102
  Loading Data...
  Creating Truth Masks...
  Selecting Data...
  Outputing data...
  Patient LUNG1-102 completed in 9 seconds
  Moving on.
Patient: LUNG1-103
  Loading Data...
  Creating Truth Masks...
  Selecting Data...
  Outputing data...
  Patient LUNG1-103 completed in 7 seconds
  Moving on.
Patient: LUNG1-104
  Loading Data...
  Creating Truth Masks...
  Selecting Data...
  Outputing data...
  Patient LUNG1-104 completed in 21 seconds
  Moving on.
Patient: LUNG1-105
  Loading Data...
  Creating Truth Masks...
  Selecting Data...
  Outputing data...
  Patient LUNG1-105 completed in 11 seconds
  Moving on.
Patient: LUNG1-106
  Loading Data...
  Creating Truth Masks...
  Selecting Data...
  Outputing data...
  Patient LUNG1-106 completed in 5 seconds
  Moving on.
Patient: LUNG1-107
  Loading Data...
  Creat